# 如何解析JSON输出

虽然一些模型提供商支持[返回结构化输出的内置方法](/docs/how_to/structured_output)，但并非全部都支持。我们可以使用输出解析器来帮助用户通过提示指定任意的JSON模式，查询符合该模式的模型输出，最后将该模式解析为JSON。

:::{.callout-note}
请注意，大语言模型是存在泄漏的抽象！您需要使用具有足够能力的LLM来生成格式良好的JSON。
:::

:::info 前提条件

本指南假定您已熟悉以下概念：

- [聊天模型](/docs/concepts/chat_models)
- [输出解析器](/docs/concepts/output_parsers)
- [提示模板](/docs/concepts/prompt_templates)
- [结构化输出](/docs/how_to/structured_output)
- [将可运行对象串联在一起](/docs/how_to/sequence/)

:::

[`JsonOutputParser`](https://api.js.langchain.com/classes/langchain_core.output_parsers.JsonOutputParser.html) 是一个内建选项，用于提示并解析 JSON 输出。

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs />
```

In [2]:
import { ChatOpenAI } from "@langchain/openai";
const model = new ChatOpenAI({
  model: "gpt-4o",
  temperature: 0,
})

import { JsonOutputParser } from "@langchain/core/output_parsers"
import { ChatPromptTemplate } from "@langchain/core/prompts"

// Define your desired data structure. Only used for typing the parser output.
interface Joke {
  setup: string
  punchline: string
}

// A query and format instructions used to prompt a language model.
const jokeQuery = "Tell me a joke.";
const formatInstructions = "Respond with a valid JSON object, containing two fields: 'setup' and 'punchline'."

// Set up a parser + inject instructions into the prompt template.
const parser = new JsonOutputParser<Joke>()

const prompt = ChatPromptTemplate.fromTemplate(
  "Answer the user query.\n{format_instructions}\n{query}\n"
);

const partialedPrompt = await prompt.partial({
  format_instructions: formatInstructions
});

const chain = partialedPrompt.pipe(model).pipe(parser);

await chain.invoke({ query: jokeQuery });

{
  setup: "Why don't scientists trust atoms?",
  punchline: "Because they make up everything!"
}

## 流式处理

`JsonOutputParser` 还支持流式传输部分数据块。当模型以多个数据块的形式返回部分 JSON 输出时，这会非常有用。解析器会跟踪这些部分数据块，并在模型完成输出生成时返回最终的 JSON 输出。

In [3]:
for await (const s of await chain.stream({ query: jokeQuery })) {
    console.log(s)
}

{}
{ setup: "" }
{ setup: "Why" }
{ setup: "Why don't" }
{ setup: "Why don't scientists" }
{ setup: "Why don't scientists trust" }
{ setup: "Why don't scientists trust atoms" }
{ setup: "Why don't scientists trust atoms?", punchline: "" }
{ setup: "Why don't scientists trust atoms?", punchline: "Because" }
{
  setup: "Why don't scientists trust atoms?",
  punchline: "Because they"
}
{
  setup: "Why don't scientists trust atoms?",
  punchline: "Because they make"
}
{
  setup: "Why don't scientists trust atoms?",
  punchline: "Because they make up"
}
{
  setup: "Why don't scientists trust atoms?",
  punchline: "Because they make up everything"
}
{
  setup: "Why don't scientists trust atoms?",
  punchline: "Because they make up everything!"
}


## 下一步
你现在已了解了一种提示模型返回结构化JSON的方法。接下来，请查看关于获取结构化输出的[更全面指南](/docs/how_to/structured_output)，了解其他技术。